In [7]:
import pandas as pd
import numpy as np
from collections import defaultdict
import nltk
import string
from nltk.stem.porter import *
from sklearn import linear_model
import sys
import unicodedata
import string
import operator
import math
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from scipy import sparse
from scipy.spatial.distance import cosine
from sklearn.metrics import average_precision_score
from sklearn.metrics import precision_recall_curve
import matplotlib.pyplot as plt
stdout = sys.stdout
reload(sys)  
sys.setdefaultencoding('utf8')
sys.stdout = stdout

In [4]:
import jsonlines
import numpy as np
import cPickle as pickle
import os

class Preprocessor:
    def __init__(self, dataDir = './'):
        self.data = []
        self.dataDir = dataDir 

    def isSampleValid(self, sample):
        return sample['gold_label'] != '-'
    
    def getDataFromJSONL(self, filename):
        numInvalid = 0
        with open(filename) as fp:
            reader = jsonlines.Reader(fp)
            for obj in reader.iter(type=dict, skip_invalid=True):
                if self.isSampleValid(obj):
                    sample = {}
                    sample['gold_label'] = obj['gold_label']
                    sample['sentence1'] = obj['sentence1']
                    sample['sentence2'] = obj['sentence2']
                    self.data.append(sample)
                else:
                    numInvalid+=1
        print len(self.data)
        print numInvalid
    
    

In [5]:
pp = Preprocessor()
pp.getDataFromJSONL('./snli_1.0_train.jsonl')

549367
785


In [62]:
print pp.data[4]

{'gold_label': u'entailment', 'sentence1': u'Children smiling and waving at camera', 'sentence2': u'There are children present'}


In [63]:
#nltk.download("stopwords")
reload(nltk)
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.translate.bleu_score import sentence_bleu
stop = set(stopwords.words('english'))
sentence1 = pp.data[4]['sentence1']
sentence1 = unicodedata.normalize('NFKD', sentence1).encode('ascii','ignore')
reference = [[i for i in sentence1.lower().split() if i not in stop]]
sentence2 = pp.data[4]['sentence2']
sentence2 = unicodedata.normalize('NFKD', sentence2).encode('ascii','ignore')
candidate = [i for i in sentence2.lower().split() if i not in stop]
length = min([len(reference[0]), len(candidate)]) 
if length < 4:
    weights = ( 1. / length ,) * length
print reference
print candidate
score = sentence_bleu(reference, candidate, weights)
print(score)

[['children', 'smiling', 'waving', 'camera']]
['children', 'present']
0.260130047511


In [52]:
# prefect match
reload(nltk)
import nltk
from nltk.translate.bleu_score import sentence_bleu
reference = [['children', 'smiling', 'waving', 'camera']]
candidate = ['children', 'smiling', 'waving']
length = min([len(reference[0]), len(candidate)]) 
weights = ( 1. / length ,) * length
score = sentence_bleu(reference, candidate, weights)
print(score)

0.716531310574


3
